# Finite State Transducers
Dartmouth College, LING48, Spring 2024<br>
Samuel Peter (samuel.peter.25@dartmouth.edu)<br>

This program uses the `openfst` and `graphviz` packages:<br>
http://www.openfst.org/twiki/bin/view/FST/PythonExtension<br>
https://graphviz.readthedocs.io/en/stable/manual.html

In this small example, a transductor reads Bribri words and decomposes them morphologically. For example (in English):

>cats -> cat-PL<br>
>dogs -> dog-PL<br>
>cities -> city-PL

This program has four parts:

(1) First, we have the `symbols`. This is the list of all of the elements you are going to have in the FST. (Practical advice: Make your transitions on paper first, and THEN figure out the symbols as you go).

(2) Second, you have the `paths`. This is the list of all the transitions and final states in the FST.

(3) These are sent to a function called `spellout`. This function has the string as its input, and then goes character by character, calculating the path through the FST and its corresponding transformations. (You do not need to modify this).

(4) Finally, we have a function that prints the FST into a PDF so you can see the transitions graphically. (You do not need to modify this).

## Step 1: Load packages and get FST compilation code

In [34]:
# Download the code to compile the FST
!gdown 1FPs__pexWcLFXH4d8ybDeoUYFfJYa5ay -O /content/hw2-fst.py

Downloading...
From (original): https://drive.google.com/uc?id=1FPs__pexWcLFXH4d8ybDeoUYFfJYa5ay
From (redirected): https://drive.google.com/uc?id=1FPs__pexWcLFXH4d8ybDeoUYFfJYa5ay&confirm=t&uuid=dd324573-abe4-465a-a5db-aced7eb5d966
To: /content/hw2-fst.py
100% 4.82k/4.82k [00:00<00:00, 16.1MB/s]


In [35]:
# This will install Python 3.7 in the virtual computer.
# This is needed to run the FST packages
# This should take about 40 seconds

%env PYTHONPATH=
!echo $PYTHONPATH

! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.12.0-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.12.0-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.12.0-Linux-x86_64.sh -b -f -p /usr/local/

import sys
sys.path.append("/usr/local/lib/python3.7/site-packages")
!conda config --add channels bioconda
!conda config --add channels conda-forge

env: PYTHONPATH=

--2024-04-12 04:01:11--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.12.0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.191.158, 104.16.32.241, 2606:4700::6810:20f1, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.191.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104996770 (100M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.12.0-Linux-x86_64.sh.1’

Miniconda3-py37_4.1 100%[===================>] 100.13M   153MB/s    in 0.7s    

2024-04-12 04:01:12 (153 MB/s) - ‘Miniconda3-py37_4.12.0-Linux-x86_64.sh.1’ saved [104996770/104996770]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ | / done

# All requested packages already installed.

installation finished.


In [36]:
# Install the packages needed to build and print the FST
!pip install openfst-python
!pip install graphviz

In [37]:
# A helper function to save a list into a text file
def saveListAsFile(inputList, filename):

  output = ""
  for l in inputList:
    output = output+l+"\n"
  output = output[:-1]

  f = open(filename, "w")
  f.write(output)
  f.close()

## Step 2: Define the symbols of the FST

The first element is a symbol in the FST (it can be one or more characters). The second element must be a number.

In [38]:
morphSymbols = []
translateSymbols = []

#add all unique symbols needed to complete the morphsymbols FST
morphSymbols.append("<eps> 0")
morphSymbols.append("a 1")
morphSymbols.append("l- 2")
morphSymbols.append("e 3")
morphSymbols.append("H 4")
morphSymbols.append("i 5")
morphSymbols.append("' 6")
morphSymbols.append("o 7")
morphSymbols.append("q 8")
morphSymbols.append("F 9")
morphSymbols.append("k 10")
morphSymbols.append("u 11")
morphSymbols.append("x 12")
morphSymbols.append("r 13")
morphSymbols.append("n- 14")
morphSymbols.append("l 15")
morphSymbols.append("n 16")
morphSymbols.append("t 17")
morphSymbols.append("s 18")
morphSymbols.append("k- 19")
morphSymbols.append("b 20")
morphSymbols.append("i- 21")


#add all unique symbols needed to complete the translatesymbols FST
translateSymbols.append("<eps> 0")
translateSymbols.append("cook- 1")
translateSymbols.append("IPFV 2")
translateSymbols.append("INF 3")
translateSymbols.append("IMP 4")
translateSymbols.append("THEME. 5")
translateSymbols.append("PFV.IMPROSP 6")
translateSymbols.append("DESIDERATIVE 7")
translateSymbols.append("MID 8")
translateSymbols.append("-PFV.IMPROSP 9")
translateSymbols.append("-INF 10")
translateSymbols.append("a 11")
translateSymbols.append("l 12")
translateSymbols.append("e 13")
translateSymbols.append("H 14")
translateSymbols.append("i 15")
translateSymbols.append("' 16")
translateSymbols.append("o 17")
translateSymbols.append("q 18")
translateSymbols.append("F 19")
translateSymbols.append("k 20")
translateSymbols.append("u 21")
translateSymbols.append("x 22")
translateSymbols.append("r 23")
translateSymbols.append("n 24")
translateSymbols.append(".IPFV 25")
translateSymbols.append("dig- 26")
translateSymbols.append("pop- 27")
translateSymbols.append("t 28")
translateSymbols.append("s 29")
translateSymbols.append("k 30")
translateSymbols.append("b 31")

## Step 3: Define the transitions between the states

The first number is the identifier of the start state. The second number is the identifier of the end state.

The first sequence of characters is the input symbol. The second sequence of characters is the output symbol.

In [39]:
mPaths = []   # Morpheme transition paths
tPaths = []   # Translation transition paths

mPaths.append("0 1 a a")           # Transition from state 0 to state 1. You receive an 'a' and return an 'a'
mPaths.append("1 2 l l-")          # Transition from state 1 to state 2. You receive an 'l' and return an 'l-'
mPaths.append("2 3 e e")           # Transition from state 2 to state 3. You receive an 'e' and return an 'e'
mPaths.append("3 4 H H")           # Transition from state 3 to state 4. You receive an 'H' and return an 'H'
mPaths.append("4")                 # State 4 is an end state. The FST finished recognizing the word "aleH"
mPaths.append("2 5 i i")           # Transition from state 2 to state 5. You receive an 'i' and return an 'i'
mPaths.append("5 6 ' '")           # Transition from state 5 to state 6. You receive a ' ' and return a ' '
mPaths.append("6")                 # State 6 is an end state. The FST finished recognizing the word "ali'"
mPaths.append("2 7 o o")           # Transition from state 2 to state 7. You receive an 'o' and return an 'o'
mPaths.append("7 8 q q")           # Transition from state 7 to state 8. You receive a 'q' and return a 'q'
mPaths.append("8 9 F F")           # Transition from state 8 to state 9. You receive an 'F' and return an 'F'
mPaths.append("9 10 k k")          # Transition from state 9 to state 10. You receive a 'k' and return a 'k'
mPaths.append("10")                # State 10 is an end state. The FST finished recognizing the word "aloqFk"
mPaths.append("9 11 <eps> <eps>")  # Transition from state 9 to state 11. You receive an '<eps>' and return an '<eps>'
mPaths.append("11")                # State 11 is an end state. The FST finished recognizing the word "aloqF"
mPaths.append("2 12 a a")          # Transition from state 2 to state 12. You receive an 'a' and return an 'a'
mPaths.append("12 13 ' '")         # Transition from state 12 to state 13. You receive a ' ' and return a ' '
mPaths.append("13 14 k k")         # Transition from state 13 to state 14. You receive a 'k' and return a 'k'
mPaths.append("14 15 u u")         # Transition from state 14 to state 15. You receive an 'u' and return an 'u'
mPaths.append("15 16 x x")         # Transition from state 15 to state 16. You receive an 'x' and return an 'x'
mPaths.append("16")                # State 16 is an end state. The FST finished recognizing the word "ala'kux"
mPaths.append("5 17 H H")          # Transition from state 5 to state 17. You receive an 'H' and return an 'H'
mPaths.append("17 18 r r")         # Transition from state 17 to state 18. You receive an 'r' and return an 'r'
mPaths.append("18")                # State 18 is an end state. The FST finished recognizing the word "aliHr"
mPaths.append("17 19 n n-")        # Transition from state 17 to state 19. You receive an 'n' and return an 'n-'
mPaths.append("19 20 e e")         # Transition from state 19 to state 20. You receive an 'e' and return an 'e'
mPaths.append("20 21 x x")         # Transition from state 20 to state 21. You receive an 'x' and return an 'x'
mPaths.append("21")                # State 21 is an end state. The FST finished recognizing the word "aliHnex"
mPaths.append("19 22 u u")         # Transition from state 19 to state 22. You receive a 'u' and return a 'u'
mPaths.append("22 23 x x")         # Transition from state 22 to state 23. You receive an 'x' and return an 'x'
mPaths.append("23 24 k k")         # Transition from state 23 to state 24. You receive a 'k' and return a 'k'
mPaths.append("24")                # State 24 is an end state. The FST finished recognizing the word "aliHnuxk"
###################################################################################################################
mPaths.append("0 25 t t")         # Transition from state 23 to state 24. You receive a 'k' and return a 'k'
mPaths.append("25 26 s s")         # Transition from state 23 to state 24. You receive a 'k' and return a 'k'
mPaths.append("26 27 a a")         # Transition from state 23 to state 24. You receive a 'k' and return a 'k'
mPaths.append("27 2 k k-")         # Transition from state 23 to state 24. You receive a 'k' and return a 'k'
mPaths.append("0 28 b b")         # Transition from state 23 to state 24. You receive a 'k' and return a 'k'
mPaths.append("28 2 i i-")         # Transition from state 23 to state 24. You receive a 'k' and return a 'k'



tPaths.append("0 1 a <eps>")           # Transition from state 0 to state 1. You receive a 'a' and return a epsilon.
tPaths.append("1 2 l cook-")           # Transition from state 1 to state 2. You receive an 'l' and return a 'cook-'
tPaths.append("2 3 e <eps>")           # Transition from state 2 to state 3. You receive a 'e' and return a epsilon.
tPaths.append("3 4 H IPFV")            # Transition from state 3 to state 4. You receive a 'H' and return 'IPFV'.
tPaths.append("4")                 # State 4 is an end state. The FST finished recognizing the word "cook-IPFV"
tPaths.append("2 7 o <eps>")           # Transition from state 2 to state 7. You receive a '0' and return a epsilon
tPaths.append("7 8 q <eps>")           # Transition from state 7 to state 8. You receive a 'q' and return a epsilon
tPaths.append("8 9 F <eps>")           # Transition from state 8 to state 9. You receive a 'F' and return a epsilon
tPaths.append("9 10 k INF")           # Transition from state 9 to state 10. You receive a 'k' and return a INF
tPaths.append("10")                 # State 10 is an end state. The FST finished recognizing the word cook-INF
tPaths.append("9 11 <eps> IMP")           # Transition from state 9 to state 11. You receive a epsilon and return a epsilon
tPaths.append("11")                 # State 11 is an end state. The FST finished recognizing the word cook-IMP
tPaths.append("2 12 a <eps>")           # Transition from state 2 to state 12. You receive a 'a' and return a epsilon
tPaths.append("12 13 ' <eps>")           # Transition from state 12 to state 13. You receive a ''' and return a epsilon
tPaths.append("13 14 k <eps>")           # Transition from state 13 to state 14. You receive a 'k' and return a epsilon
tPaths.append("14 15 u <eps>")           # Transition from state 14 to state 15. You receive a 'u' and return a epsilon
tPaths.append("15 16 x DESIDERATIVE")           # Transition from state 15 to state 16. You receive a 'x' and return DESIDERATIVE
tPaths.append("16")                 # State 16 is an end state. The FST finished recognizing the word cook-DESIDERATIVE
tPaths.append("2 5 i THEME.")           # Transition from state 2 to state 5. You receive a 'i' and return THEME.
tPaths.append("5 6 ' PFV.IMPROSP")           # Transition from state 5 to state 6. You receive a ''' and return PFV.IMPROSP
tPaths.append("6")                 # State 6 is an end state. The FST finished recognizing the word cook-PFV.IMPROSP
tPaths.append("5 17 H MID")           # Transition from state 5 to state 17. You receive a 'H' and return MID
tPaths.append("17 18 r .IPFV")           # Transition from state 17 to state 18. You receive a 'r' and return .IPFV
tPaths.append("18")                 # State 18 is an end state. The FST finished recognizing the word cook-THEME.MID.IPFV
tPaths.append("17 19 n <eps>")           # Transition from state 17 to state 19. You receive a 'n' and return epsilon
tPaths.append("19 20 e <eps>")           # Transition from state 19 to state 20. You receive a 't' and return epsilon
tPaths.append("20 21 x -PFV.IMPROSP")           # Transition from state 20 to state 21. You receive a 'x' and return -PFV.IMPROSP
tPaths.append("21")                 # State 21 is an end state. The FST finished recognizing the word cook-THEME.MID-PFV.IMPROSP
tPaths.append("19 22 u <eps>")           # Transition from state 19 to state 22. You receive a 'u' and return epsilon
tPaths.append("22 23 x <eps>")           # Transition from state 22 to state 23. You receive a 'x' and return epsilon
tPaths.append("23 24 k -INF")           # Transition from state 23 to state 24. You receive a 'k' and return _INF
tPaths.append("24")                 # State 24 is an end state. The FST finished recognizing the word cook-THEME.MID-INF
################################################################################################################################
tPaths.append("0 25 t <eps>")           # Transition from state 23 to state 24. You receive a 'k' and return _INF
tPaths.append("25 26 s <eps>")           # Transition from state 23 to state 24. You receive a 'k' and return _INF
tPaths.append("26 27 a <eps>")           # Transition from state 23 to state 24. You receive a 'k' and return _INF
tPaths.append("27 2 k pop-")           # Transition from state 23 to state 24. You receive a 'k' and return _INF
tPaths.append("0 28 b <eps>")           # Transition from state 23 to state 24. You receive a 'k' and return _INF
tPaths.append("28 2 i dig-")           # Transition from state 23 to state 24. You receive a 'k' and return _INF


In [40]:
# Save the symbols and transitions into files
saveListAsFile(morphSymbols, "/content/morphSymbols.txt")
saveListAsFile(translateSymbols, "/content/translateSymbols.txt")
saveListAsFile(mPaths, "/content/mPaths.txt")
saveListAsFile(tPaths, "/content/tPaths.txt")

In [41]:
# This function returns three variables:
# (1) The word itself
# (2) The word split by morphemes
# (3) The word with the translation of its morphemes

import os.path

def getAnalysis(word):

  if (os.path.isfile("/content/output-split.txt")):
    !rm output-split.txt
  if (os.path.isfile("/content/output-translation.txt")):
    !rm output-translation.txt
  if (os.path.isfile("/content/wordToAnalyze.txt")):
    !rm wordToAnalyze.txt

  f = open("wordToAnalyze.txt", "w")
  f.write(word)
  f.close()

  !python hw2-fst.py

  try: splitWord = open("output-split.txt", "r").read()
  except: splitWord = ""

  try: wordTranslation = open("output-translation.txt", "r").read()
  except: wordTranslation = ""

  return(word, splitWord, wordTranslation)

## Step 4: Compile and query the FST

Here are a few examples of how you can get an analysis from the FST.

In [42]:
word = "aleH"
print(getAnalysis(word))

('aleH', 'al-eH', 'cook-IPFV')


In [43]:
word = "ali'"
print(getAnalysis(word))

("ali'", "al-i'", 'cook-THEME.PFV.IMPROSP')


In [44]:
word = "aloqFk"
print(getAnalysis(word))

('aloqFk', 'al-oqFk', 'cook-INF')


In [45]:
word = "aloqF"
print(getAnalysis(word))

('aloqF', 'al-oqF', 'cook-IMP')


In [46]:
word = "ala'kux"
print(getAnalysis(word))

("ala'kux", "al-a'kux", 'cook-DESIDERATIVE')


In [47]:
word = "aliHr"
print(getAnalysis(word))

('aliHr', 'al-iHr', 'cook-THEME.MID.IPFV')


In [48]:
word = "aliHnex"
print(getAnalysis(word))

('aliHnex', 'al-iHn-ex', 'cook-THEME.MID-PFV.IMPROSP')


In [49]:
word = "aliHnuxk"
print(getAnalysis(word))

('aliHnuxk', 'al-iHn-uxk', 'cook-THEME.MID-INF')


In [50]:
word = "tsakoqFk"
print(getAnalysis(word))

('tsakoqFk', 'tsak-oqFk', 'pop-INF')


In [51]:
word = "bioqFk"
print(getAnalysis(word))

('bioqFk', 'bi-oqFk', 'dig-INF')


## Optional: Download the FSTs

The Python file that compiles the FST created a PDF file with the FST. You can download them here.

In [52]:
from google.colab import files

In [53]:
files.download('morph.gv.pdf')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [54]:
files.download('translate.gv.pdf')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>